In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Resize, Compose

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cuda


In [23]:
# Define the CNN architecture


class Net(nn.Module):
    def __init__(self):
        """
        CNN Architecture: 2 convolutional layers, 1 max pool, 2 fully connected layer, 
        and final fully connected layer to classify 10 different classes
        """
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        """
        Defines the forward pass of the network.
        """
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x


In [24]:
data_dir = "/home/bishoy/Fourth Year/Second Semester/Satellite Imagery/SI_Project/datasetResNet/train"
transformation = Compose([Resize((64, 64)), ToTensor()])
dataset = ImageFolder(data_dir, transform=transformation)


In [25]:
# use the whole dataset to train the model
train_set = dataset
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=4, shuffle=True)
print("Number of training samples: ", len(train_set))


Number of training samples:  732


In [28]:
# Define the loss function and optimizer
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [29]:
# Train the network
for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        # move the data to the GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')


[1,   100] loss: 0.697
[2,   100] loss: 0.692
[3,   100] loss: 0.688
[4,   100] loss: 0.673
[5,   100] loss: 0.648
[6,   100] loss: 0.636
[7,   100] loss: 0.608
[8,   100] loss: 0.599
[9,   100] loss: 0.577
[10,   100] loss: 0.566
[11,   100] loss: 0.568
[12,   100] loss: 0.544
[13,   100] loss: 0.531
[14,   100] loss: 0.475
[15,   100] loss: 0.491
[16,   100] loss: 0.393
[17,   100] loss: 0.427
[18,   100] loss: 0.418
[19,   100] loss: 0.431
[20,   100] loss: 0.382
[21,   100] loss: 0.344
[22,   100] loss: 0.320
[23,   100] loss: 0.286
[24,   100] loss: 0.261
[25,   100] loss: 0.248
Finished Training


In [ ]:
# Test the network on the test data
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in test_loader:
#         images, labels = data
#         # move the data to the GPU
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = net(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))


In [30]:
# save the current model
PATH = './cnn_v3.pth'
torch.save(net.state_dict(), PATH)


In [31]:
# make new data loader for the test data
data_dir = "/home/bishoy/Fourth Year/Second Semester/Satellite Imagery/SI_Project/datasetResNet/validation"
transformation = Compose([Resize((64, 64)), ToTensor()])
dataset = ImageFolder(data_dir, transform=transformation)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=4, shuffle=True, num_workers=2)


In [32]:
# load model cnn_v2.pth
mynet = Net()
mynet = mynet.to(device)
mynet.load_state_dict(torch.load('./cnn_v3.pth'))
mynet.eval()

# use the model to predict the class of a random image
# Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # move the data to the GPU
        images = images.to(device)
        labels = labels.to(device)
        outputs = mynet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test images: 78 %
